## 1. Binder 通信简述
- Client 进程通过 RPC(Remote Procedure Call Protocol) 与 Server 通信，可以简单地划分为三层，驱动层、IPC 层、业务层。demo() 便是Client 端和 Server 共同协商好的统一方法；handle、RPC 数据、代码、协议这4项组成了 IPC 层的数据，通过 IPC 层进行数据传输；而真正在Client 和Server 两端建立通信的基础设施便是 Binder Driver。

![image](binder2_page1.png)

- 例如，当名为 BatteryStatsService 的 Client 向 ServiceManager 注册服务的过程中，IPC 层的数据组成为：Handle=0，RPC 代码为ADD_SERVICE_TRANSACTION，RPC 数据为 BatteryStatsService，Binder 协议为 BC_TRANSACTION。

---

## 2. Binder 通信协议
### 2.1 通信模型
- 先列举一次完整的 Binder 通信过程：
![image](binder2_page2.png)

- Binder 协议包含在 IPC 数据中，分为两类:
  1. BINDER_COMMAND_PROTOCOL：binder 请求码，以”BC_“开头，简称 BC 码，用于从 IPC 层传递到 Binder Driver 层
  2. BINDER_RETURN_PROTOCOL ：binder 响应码，以”BR_“开头，简称 BR 码，用于从 Binder Driver 层传递到 IPC 层

- Binder IPC 通信至少是两个进程的交互：
  - client 进程执行 binder_thread_write，根据 BC_XXX 命令，生成相应的 binder_work
  - server 进程执行 binder_thread_read，根据 binder_work.type 类型，生成 BR_XXX，发送到用户空间处理
  
#### 2.1.1 通信过程
![image](binder2_page3.png)

- 其中 binder_work.type 共有6种类型：
```
BINDER_WORK_TRANSACTION   // 最常见类型
BINDER_WORK_TRANSACTION_COMPLETE
BINDER_WORK_NODE
BINDER_WORK_DEAD_BINDER
BINDER_WORK_DEAD_BINDER_AND_CLEAR
BINDER_WORK_CLEAR_DEATH_NOTIFICATION
```

---

### 2.2 binder_thread_write
- 请求处理过程是通过 binder_thread_write() 方法，该方法用于处理 Binder 协议中的请求码。当 binder_buffer 存在数据，binder 线程的写操作循环执行。

```
binder_thread_write() {
    while (ptr < end && thread->return_error == BR_OK) {
        get_user(cmd, (uint32_t __user *)ptr)；// 获取 IPC 数据中的 Binder 协议(BC码)
        switch (cmd) {
            case BC_INCREFS: ...
            case BC_ACQUIRE: ...
            case BC_RELEASE: ...
            case BC_DECREFS: ...
            case BC_INCREFS_DONE: ...
            case BC_ACQUIRE_DONE: ...
            case BC_FREE_BUFFER: ... break;
            
            case BC_TRANSACTION:
            case BC_REPLY: {
                struct binder_transaction_data tr;
                copy_from_user(&tr, ptr, sizeof(tr))； // 拷贝用户空间 tr 到内核
                // 【见小节2.2.1】
                binder_transaction(proc, thread, &tr, cmd == BC_REPLY);
                break;

            case BC_REGISTER_LOOPER: ...
            case BC_ENTER_LOOPER: ...
            case BC_EXIT_LOOPER: ...
            case BC_REQUEST_DEATH_NOTIFICATION: ...
            case BC_CLEAR_DEATH_NOTIFICATION:  ...
            case BC_DEAD_BINDER_DONE: ...
            }
        }
    }
}
```

- 对于请求码为 BC_TRANSACTION 或 BC_REPLY 时，会执行 binder_transaction() 方法，这是最为频繁的操作。对于其他命令则不同。

#### 2.2.1 binder_transaction
```
static void binder_transaction(struct binder_proc *proc,
               struct binder_thread *thread,
               struct binder_transaction_data *tr, int reply){
    //根据各种判定，获取以下信息：
    struct binder_thread *target_thread； //目标线程
    struct binder_proc *target_proc；    //目标进程
    struct binder_node *target_node；    //目标binder节点
    struct list_head *target_list；      //目标TODO队列
    wait_queue_head_t *target_wait；     //目标等待队列
    ...
    
    //分配两个结构体内存
    struct binder_transaction *t = kzalloc(sizeof(*t), GFP_KERNEL);
    struct binder_work *tcomplete = kzalloc(sizeof(*tcomplete), GFP_KERNEL);
    //从target_proc分配一块buffer【见小节3.2】
    t->buffer = binder_alloc_buf(target_proc, tr->data_size,

    for (; offp < off_end; offp++) {
        switch (fp->type) {
        case BINDER_TYPE_BINDER: ...
        case BINDER_TYPE_WEAK_BINDER: ...
        case BINDER_TYPE_HANDLE: ...
        case BINDER_TYPE_WEAK_HANDLE: ...
        case BINDER_TYPE_FD: ...
        }
    }
    //向目标进程的target_list添加BINDER_WORK_TRANSACTION事务
    t->work.type = BINDER_WORK_TRANSACTION;
    list_add_tail(&t->work.entry, target_list);
    //向当前线程的todo队列添加BINDER_WORK_TRANSACTION_COMPLETE事务
    tcomplete->type = BINDER_WORK_TRANSACTION_COMPLETE;
    list_add_tail(&tcomplete->entry, &thread->todo);
    if (target_wait)
        wake_up_interruptible(target_wait);
    return;
}
```

- 路由过程：handle -> ref -> target_node -> target_proc
  - reply 的过程会找到 target_thread
  - 非 reply 则一般找到 target_proc
  - 对特殊的嵌套 binder call 会根据 transaction_stack 来决定是插入事务到目标线程还是目标进程

#### 2.2.2 BC_PROTOCOL
- binder 请求码，是用 enum binder_driver_command_protocol 来定义的，是用于应用程序向 binder 驱动设备发送请求消息，应用程序包含 Client 端和 Server 端，以 BC_ 开头

- BC_FREE_BUFFER
  - 通过 mmap() 映射内存，其中 ServiceManager 映射的空间大小为 128K，其他 Binder 应用进程映射的内存大小为 1M-8K
  - Binder 驱动基于这块映射的内存采用最佳匹配算法来动态分配和释放，通过 binder_buffer 结构体中的 free 字段来表示相应的 buffer 是空闲还是已分配状态。对于已分配的 buffers 加入到 binder_proc 中的 allocated_buffers 红黑树; 对于空闲的 buffers 加入到 binder_proc 中的 free_buffers 红黑树
  - 当应用程序需要内存时，根据所需内存大小从 free_buffers 中找到最合适的内存，并放入 allocated_buffers 树；当应用程序处理完后必须尽快使用 BC_FREE_BUFFER 命令来释放该 buffer，从而添加回到 free_buffers 树。
- BC_INCREFS、BC_ACQUIRE、BC_RELEASE、BC_DECREFS 等请求码的作用是对 binder 的强/弱引用的计数操作，用于实现强/弱指针的功能。
- 参数类型主要有以下几类：
  - binder_transaction_data（结构体）
  - binder_ptr_cookie（由binder指针和cookie组成）
  - binder_uintptr_t（指针）
  - __u32（无符号整型）
- Binder 线程创建与退出：
  - BC_ENTER_LOOPER：binder 主线程(由应用层发起)的创建会向驱动发送该消息；joinThreadPool() 过程创建 binder 主线程
  - BC_REGISTER_LOOPER：Binder 用于驱动层决策而创建新的 binder 线程；joinThreadPool() 过程,创建非 binder 主线程
  - BC_EXIT_LOOPER：退出 Binder 线程，对于 binder 主线程是不能退出; joinThreadPool() 的过程出现 timeout, 并且非 binder 主线程,则会退出该 binder 线程

---

### 2.3 binder_thread_read
- 响应处理过程是通过 binder_thread_read() 方法，该方法根据不同的 binder_work->type 以及不同状态，生成相应的响应码。

```
binder_thread_read（）{
    wait_for_proc_work = thread->transaction_stack == NULL &&
            list_empty(&thread->todo);
    //根据wait_for_proc_work来决定wait在当前线程还是进程的等待队列
    if (wait_for_proc_work) {
        ret = wait_event_freezable_exclusive(proc->wait, binder_has_proc_work(proc, thread));
        ...
    } else {
        ret = wait_event_freezable(thread->wait, binder_has_thread_work(thread));
        ...
    }
    
    while (1) {
        //当&thread->todo和&proc->todo都为空时，goto到retry标志处，否则往下执行：
        struct binder_transaction_data tr;
        struct binder_transaction *t = NULL;
        switch (w->type) {
          case BINDER_WORK_TRANSACTION: ...
          case BINDER_WORK_TRANSACTION_COMPLETE: ...
          case BINDER_WORK_NODE: ...
          case BINDER_WORK_DEAD_BINDER: ...
          case BINDER_WORK_DEAD_BINDER_AND_CLEAR: ...
          case BINDER_WORK_CLEAR_DEATH_NOTIFICATION: ...
        }
        ...
    }
done:
    *consumed = ptr - buffer;
    //当满足请求线程加已准备线程数等于0，已启动线程数小于最大线程数(15)，
    //且looper状态为已注册或已进入时创建新的线程。
    if (proc->requested_threads + proc->ready_threads == 0 &&
        proc->requested_threads_started < proc->max_threads &&
        (thread->looper & (BINDER_LOOPER_STATE_REGISTERED |
         BINDER_LOOPER_STATE_ENTERED))) {
        proc->requested_threads++;
        // 生成BR_SPAWN_LOOPER命令，用于创建新的线程
        put_user(BR_SPAWN_LOOPER, (uint32_t __user *)buffer)；
    }
    return 0;
}
```

- 当 transaction 堆栈为空，且线程 todo 链表为空，且 non_block=false 时，意味着没有任何事务需要处理的，会进入等待客户端请求的状态。当有事务需要处理时便会进入循环处理过程，并生成相应的响应码。在 Binder 驱动层，只有在进入 binder_thread_read() 方法时，同时满足以下条件， 才会生成 BR_SPAWN_LOOPER 命令，当用户态进程收到该命令则会创建新线程：
  - binder_proc 的 requested_threads 线程数为 0
  - binder_proc 的 ready_threads 线程数为 0
  - binder_proc 的 requested_threads_started 个数小于 15(即最大线程个数)
  - binder_thread 的 looper 状态为 BINDER_LOOPER_STATE_REGISTERED 或 BINDER_LOOPER_STATE_ENTERED
  
#### 2.3.1 BR_PROTOCOL
- binder 响应码，是用 enum binder_driver_return_protocol 来定义的，是 binder 设备向应用程序回复的消息，，应用程序包含 Client 端和 Server 端，以 BR_ 开头，总 18 条；

- BR_SPAWN_LOOPER：binder 驱动已经检测到进程中没有线程等待即将到来的事务。那么当一个进程接收到这条命令时，该进程必须创建一条新的服务线程并注册该线程，在接下来的响应过程会看到何时生成该响应码。
- BR_TRANSACTION_COMPLETE：当 Client 端向 Binder 驱动发送 BC_TRANSACTION 命令后，Client 会收到 BR_TRANSACTION_COMPLETE 命令，告知 Client 端请求命令发送成功；对于 Server 向 Binder 驱动发送 BC_REPLY 命令后，Server 端会收到 BR_TRANSACTION_COMPLETE 命令，告知 Server 端请求回应命令发送成功。
- BR_DEAD_REPLY: 当应用层向 Binder 驱动发送 Binder 调用时，若 Binder 应用层的另一个端已经死亡，则驱动回应 BR_DEAD_BINDER 命令。
- BR_FAILED_REPLY: 当应用层向 Binder 驱动发送 Binder 调用时，若 transaction 出错，比如调用的函数号不存在，则驱动回应BR_FAILED_REPLY。

---